### Keras CNN 实现 CHF 判别

#### 1. 使用 conda 安装 Keras

` conda install keras `

#### 2. 导入 CNF 数据集

In [1]:
from PIL import Image
import os
import numpy

output_dir = 'output'

font_len = 13
characters_count = 40

images = []
label = []

for i, output_file in enumerate([name for name in os.listdir(output_dir) if name != '.DS_Store']):
    for file_name in [name for name in os.listdir(output_dir+os.sep+output_file) if name != '.DS_Store']:
        images.append(list(Image.open(output_dir+os.sep+output_file+os.sep+file_name).getdata()))
        label.append(i)

x=numpy.array(images)
y=numpy.array(label)

#### 3. 处理数据集

* 测试模式下，有 40 个字，每种字套用 13 种字体

* 除以 255 是为了将 0-255 的 LA 值映射到 0-1 之间

In [2]:
x = x.reshape(x.shape[0],128,128,2).astype('float32') / 255

In [3]:
from keras.utils import np_utils
y = np_utils.to_categorical(y)

Using TensorFlow backend.


#### 4. 建立模型

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten

In [5]:
model = Sequential([
    Conv2D(input_shape=(128,128,2),
           filters=12,
           kernel_size=3,
           padding='same',
           activation='relu'),
    Dropout(0.25),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=24,
           kernel_size=3,
           padding='same',
           activation='relu'),
    Dropout(0.25),
    Conv2D(filters=48,
           kernel_size=3,
           padding='same',
           activation='relu'),
    Dropout(0.25),
    Flatten(),
    Dense(units=1600,
          kernel_initializer='normal',
          activation='relu'),
    Dropout(0.5),
    Dense(units=400,
          kernel_initializer='normal',
          activation='relu'),
    Dense(units=40,
          kernel_initializer='normal',
          activation='softmax')
])

#### 5. 输出模型结构

In [6]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 12)      228       
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 128, 12)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 12)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 24)        2616      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 64, 24)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 48)        10416     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64, 64, 48)        0         
__________

#### 6. 定义训练方式

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#### 7. 开始训练

In [ ]:
history = model.fit(x=x,
                   y=y,
                   validation_split=0.2,
                   epochs=10,
                   batch_size=32,
                   verbose=2)